In [13]:
import cv2
import numpy as np

In [27]:
img = cv2.imread("mod_img3.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,th = cv2.threshold(grey, 150, 255, cv2.THRESH_BINARY)
kernel = np.ones((1,5),np.uint8)
th = cv2.erode(th, kernel, iterations = 2)
th = cv2.dilate(th, kernel, iterations = 2)
th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result

kernel = np.ones((3, 3), np.uint8)   
result = cv2.erode(result, kernel, iterations = 1)
result = cv2.dilate(result, kernel, iterations = 1)

cv2.imwrite("inter1.jpg", result)

True

In [28]:
contours, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()

#finding radio buttons 

for cnt in contours:
    approx1 = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    if len(approx1) > 10:
        cv2.drawContours(img2,[cnt],0,(0,255,255),-1)

#end

for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])


i = 0
others = list()
while i<len(boundRect):
    if boundRect[i][2] > 20 and boundRect[i][3] > 20:
        i += 1
    else:
        temp = boundRect[i]
        boundRect.remove(boundRect[i])
        if temp[2] >10 and temp[3] >5 and abs(temp[2] -temp[3])<8:
            others.append(temp)
        
boundRect.sort(key = lambda x:x[0])


i = 0

while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1
print(boundRect)

[(0, 3687, 191, 473), (0, 3590, 35, 73), (33, 3660, 38, 49), (82, 3799, 42, 49), (361, 1184, 534, 462), (423, 1236, 446, 382), (485, 1236, 44, 25), (566, 1382, 146, 119), (574, 2251, 352, 363), (606, 2281, 299, 306), (691, 2389, 108, 127), (896, 3087, 751, 1073), (977, 4056, 27, 64), (1167, 3636, 44, 31), (1186, 3495, 23, 38), (1279, 3238, 34, 42), (1312, 3054, 21, 31), (1343, 3015, 42, 97), (1352, 3110, 45, 26), (1352, 2216, 460, 279), (1353, 1247, 443, 253), (1362, 3380, 26, 28), (1380, 3226, 24, 42), (1380, 3202, 76, 36), (1396, 3016, 30, 26), (1409, 3148, 48, 32), (1434, 3520, 26, 44), (1440, 3275, 38, 22), (1468, 2961, 35, 31), (1470, 3983, 51, 44), (1471, 2997, 33, 32), (1482, 4018, 24, 31), (1488, 3388, 38, 143), (1496, 3330, 47, 28), (1508, 3934, 21, 34), (1520, 3660, 39, 39), (1525, 3592, 33, 24), (1526, 3827, 24, 26), (1531, 3786, 25, 35), (1535, 3704, 24, 22), (1538, 3213, 35, 52), (1542, 3855, 29, 30), (1598, 3421, 27, 28), (1618, 3632, 32, 60), (1635, 3551, 23, 39), (1638,

In [29]:
s = set()
for small in others:
    x, y, w, h = small[0:]
    for i in range(len(boundRect)):
        x1, y1, w1, h1 = boundRect[i][0:]
        if x >= x1+w1-h1 and x+w <= x1+w1 and y >= y1 and y+h <= y1+h1:
            cv2.rectangle(img2, (x1,y1), (x1+w1, y1+h1), (255,0,0), 2)
            cv2.rectangle(img2, (x,y), (x+w, y+h), (0,205,0), 2)
            s.add((x1,y1,w1,h1))
            break
      
#sorting the rectangle as per their height and width

def Sort_Tuple(tup):  
      
    lst = len(tup)  
    for i in range(0, lst):    
        for j in range(0, lst-i-1):  
            if (tup[j][3] < tup[j + 1][3]):  
                temp = tup[j]  
                tup[j]= tup[j + 1]  
                tup[j + 1]= temp  
    return tup 


for j in boundRect:
    if j in s:
        boundRect.remove(j)

boundRect=Sort_Tuple(boundRect)

#identifying date fields that are indented six boxes with a bigger box
  

date_fields={}
for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    if(len(date_fields)==0):
        date_fields[(x1,y1,w1,h1)]=[]
    else:
        flag=False
        for big_box in date_fields:
            x2,y2,w2,h2=big_box[0:]
            if y1>y2 and x1>x2 and y1+h1 < y2+h2 and x1+w1 < x2+w2:
                date_fields[big_box].append((x1,y1,w1,h1))
                flag=True
                break
        if flag==False:
            date_fields[(x1,y1,w1,h1)]=[]
        
print(date_fields)

inner_fields=set()

#marking date_fields and the individual boxes

for c in date_fields:
    if(len(date_fields[c])==3):
        boundRect.remove(c)
        boundRect.remove(date_fields[c][0])
        boundRect.remove(date_fields[c][1])
        boundRect.remove(date_fields[c][2])
        inner_fields.add(date_fields[c][0])
        inner_fields.add(date_fields[c][1])
        inner_fields.add(date_fields[c][2])
        cv2.rectangle(img2, (int(c[0]), int(c[1])), \
              (int(c[0]+c[2]), int(c[1]+c[3])), (247, 145, 72), 2)
    
for c in inner_fields:
    cv2.rectangle(img2, (int(c[0]), int(c[1])), \
          (int(c[0]+c[2]), int(c[1]+c[3])), (255,65,245), 2)
                
     
for c in boundRect:
    cv2.rectangle(img2, (int(c[0]), int(c[1])), \
          (int(c[0]+c[2]), int(c[1]+c[3])), (0,0,255), 2)
        
cv2.imwrite("inter6.jpg", img2)  

{(361, 1184, 534, 462): [(423, 1236, 446, 382), (566, 1382, 146, 119), (485, 1236, 44, 25)], (574, 2251, 352, 363): [(606, 2281, 299, 306), (691, 2389, 108, 127)], (1923, 1175, 137, 274): [], (1353, 1247, 443, 253): [(1743, 1354, 36, 41)], (1800, 2188, 128, 201): [], (1926, 2155, 245, 197): [(1964, 2281, 61, 41)], (1828, 1207, 132, 195): [], (1488, 3388, 38, 143): [], (1343, 3015, 42, 97): [], (0, 3590, 35, 73): [], (977, 4056, 27, 64): [], (1618, 3632, 32, 60): [], (1538, 3213, 35, 52): [], (33, 3660, 38, 49): [], (82, 3799, 42, 49): [], (1434, 3520, 26, 44): [], (1470, 3983, 51, 44): [], (1279, 3238, 34, 42): [], (1380, 3226, 24, 42): [], (1520, 3660, 39, 39): [], (1635, 3551, 23, 39): [], (1186, 3495, 23, 38): [], (1380, 3202, 76, 36): [], (1531, 3786, 25, 35): [], (1508, 3934, 21, 34): [], (1638, 3493, 26, 34): [], (1409, 3148, 48, 32): [], (1471, 2997, 33, 32): [], (1167, 3636, 44, 31): [], (1312, 3054, 21, 31): [], (1468, 2961, 35, 31): [], (1482, 4018, 24, 31): [], (1542, 3855, 

True